In [1]:
import gc
import os
import sys
import math
import random
import warnings
import pickle
from pathlib import Path
from joblib import Parallel, delayed
from statsmodels.tsa.deterministic import (CalendarFourier,
                                           CalendarSeasonality,
                                           CalendarTimeTrend,
                                           DeterministicProcess)
import pickle
import optuna
from tqdm import tqdm
import os
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
# from pandarallel import pandarallel
# pandarallel.initialize()
warnings.simplefilter("ignore")
import ctypes as ct
from datetime import timedelta

from sklearn.metrics import mean_absolute_error

import statistics as st
import lightgbm as lgbm
from scipy.stats import norm

In [2]:
sys.path.append('../../')
import src.utils as utils

## Param

In [3]:
EXP_NUM = 92
NFOLDS = 5
SEED = 42
TRAIN_DATE = 'date < 20210701'

In [4]:
# def set_seed(seed: int = 42):
#     random.seed(seed)
#     np.random.seed(seed)
#     os.environ["PYTHONHASHSEED"] = str(seed)
#     torch.manual_seed(seed)
#     torch.cuda.manual_seed(seed)  # type: ignore
#     torch.backends.cudnn.deterministic = True  # type: ignore
#     torch.backends.cudnn.benchmark = False  # type: ignore
# set_seed(SEED)

## Dir

In [5]:
DATA_DIR = Path("/home/knikaido/work/MLB-Player-Digital-Engagement-Forecasting/data/")
MAIN_DATA_DIR = DATA_DIR / 'mlb-player-digital-engagement-forecasting-update'
TRAIN_DIR = MAIN_DATA_DIR / 'train'
OUTPUT_DIR = Path('./output/')

In [6]:
players = pd.read_csv(MAIN_DATA_DIR / 'players.csv')
seasons = pd.read_csv(MAIN_DATA_DIR / 'seasons.csv')
teams = pd.read_csv(MAIN_DATA_DIR / 'teams.csv')

rosters = pd.read_csv(TRAIN_DIR / 'rosters_train.csv').query(TRAIN_DATE)
targets = pd.read_csv(TRAIN_DIR / 'nextDayPlayerEngagement_train.csv').query(TRAIN_DATE)
scores = pd.read_csv(TRAIN_DIR / 'playerBoxScores_train.csv').query(TRAIN_DATE)
scores = scores.groupby(['playerId', 'date']).sum().reset_index()
standings = pd.read_csv(TRAIN_DIR / 'standings_train.csv').query(TRAIN_DATE)
playerTwitterFollowers = pd.read_csv(TRAIN_DIR / 'playerTwitterFollowers_train.csv').query(TRAIN_DATE)
awards = pd.read_csv(TRAIN_DIR / 'awards_train.csv').query(TRAIN_DATE)


In [7]:
train_elements_dict = {"players":players, 
                       "rosters":rosters, 
                       "targets":targets, 
                       "scores":scores, 
                       "seasons":seasons, 
                       "teams":teams, 
                       "standings":standings,
                       'awards':awards}

In [8]:
def map_team_name(name):
    names = name.split('-')
    result = ''
    for n in names:
        if n == 'st':
            n = 'st.'
        result += f' {n.capitalize()}'

    return result[1:]

In [9]:
def calc_corr(df):
    # 相関係数行列を作成
    corr_mat = df.corr(method='pearson')

    # 行（列）サイズを取得
    n = corr_mat.shape[0]
    corr_ary = []

    for i in range(n):
        for j in range(i):
            if i == j:
                continue
            corr_ary.append(corr_mat.iloc[i,j])

    return corr_ary

In [10]:
def calc_probs(pid,df,temp):
    to_append=[pid,'','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','','']
    targets=['target1','target2','target3','target4']
    z=1
    for target in targets:
        target_prob = temp[target].tolist()
        mean = np.mean(target_prob)
        std = np.std(target_prob)
        median = st.median(target_prob)
        distribution = norm(mean, std)
        min_weight = min(target_prob)
        max_weight = max(target_prob)
        values = list(np.linspace(min_weight, max_weight))
        probabilities = [distribution.pdf(v) for v in values]
        max_value = max(probabilities)
        max_index = probabilities.index(max_value)
        to_append[z]=mean
        to_append[z+1]=median
        to_append[z+2]=std
        to_append[z+3]=min_weight
        to_append[z+4]=max_weight
        to_append[z+5]=temp[target].skew()
        to_append[z+6]=temp[target].kurt()

        z=z+7
    corr_ = calc_corr(temp[['target1', 'target2', 'target3', 'target4']])
    to_append[z:] = corr_  
    df_length = len(df)
    df.loc[df_length] = to_append
    return df

In [11]:
def count_consecutive_items_n_cols(df, col_name_list, output_col):
    cum_sum_list = [
        (df[col_name] != df[col_name].shift(1)).cumsum().tolist() for col_name in col_name_list
    ]
    df[output_col] = df.groupby(
        ["_".join(map(str, x)) for x in zip(*cum_sum_list)]
    ).cumcount() + 1
    return df

In [12]:
def extract_season(date_raw, season_start_end):
    idxes = 0
    for raw in season_start_end.iloc():
        idx_ = ((date_raw >= raw.iloc[0]) & (date_raw <= raw.iloc[1])) * 1
        idxes += idx_
    return idxes

In [13]:
def fit_lgbm(x_train, y_train, x_valid, y_valid, verbose=100):
    def opt(trial):
        params = {
                'random_state': SEED,
                'objective':'mae',
                'n_estimators': 10000,
                'learning_rate': 0.1,
                'max_depth': trial.suggest_int('max_depth', 1, 20),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
                'subsample': trial.suggest_discrete_uniform('subsample', 0.5, 0.9, 0.1),
                'colsample_bytree': trial.suggest_discrete_uniform('colsample_bytree', 0.5, 0.9, 0.1),
                'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-3, 1e3),
                'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-3, 1e3),
                'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
                'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
                'bagging_freq': trial.suggest_int('bagging_freq', 1, 20),
                'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
                'min_child_samples': trial.suggest_int('min_child_samples', 5, 100)
        }

        model_opt = lgbm.LGBMRegressor(**params)

        model_opt.fit(x_train, y_train, 
            eval_set=[(x_train, y_train), (x_valid, y_valid)],  
            early_stopping_rounds=verbose, 
            verbose=verbose)
        oof_pred = model_opt.predict(x_valid)
        oof_pred = np.clip(oof_pred, 0, 100)
        score = mean_absolute_error(oof_pred, y_valid)
        return -score
    return opt

In [14]:
# def fit_lgbm(x_train, y_train, x_valid, y_valid, params: dict=None, verbose=100):
#     oof_pred = np.zeros(len(y_valid), dtype=np.float32)
#     model = lgbm.LGBMRegressor(**params)
#     model.fit(x_train, y_train, 
#         eval_set=[(x_valid, y_valid)],  
#         early_stopping_rounds=verbose, 
#         verbose=verbose)
#     oof_pred = model.predict(x_valid)
#     oof_pred = np.clip(oof_pred, 0, 100)
#     score = mean_absolute_error(oof_pred, y_valid)
#     print('mae:', score)
#     return oof_pred, model, score

In [15]:
def my_timeseries_fold(train):
    
    tr_idxs = []
    val_idxs = []
        
    tr_idx = (train['date'].astype(int) < 20200801)
    val_idx = (train['date'].astype(int) >= 20200801) & (train['date'].astype(int) < 20200901)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)

    tr_idx = (train['date'].astype(int) < 20200901)
    val_idx = (train['date'].astype(int) >= 20200901) & (train['date'].astype(int) < 20201001)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)
    
    tr_idx = (train['date'].astype(int) < 20210401)
    val_idx = (train['date'].astype(int) >= 20210401) & (train['date'].astype(int) < 20210501)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)

    tr_idx = (train['date'].astype(int) < 20210501)
    val_idx = (train['date'].astype(int) >= 20210501) & (train['date'].astype(int) < 20210601)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)
    
    tr_idx = (train['date'].astype(int) < 20210601)
    val_idx = (train['date'].astype(int) >= 20210601) & (train['date'].astype(int) < 20210701)
    tr_idxs.append(tr_idx)
    val_idxs.append(val_idx)
    
    return tr_idxs, val_idxs

In [16]:
class Rt4kaidoTrain:
    def __init__(self, usetimelinefeature=False):
        
        self.usetimelinefeature = usetimelinefeature
        self.targets_cols = ['playerId', 'target1', 'target2', 'target3', 'target4', 'date']
        self.players_cols = ['playerId', 'primaryPositionName', 'birthCity', 'DOY', 'mlbDebutYear', 'mlbDebutDate', 'DebutAge', 'heightInches', 'weight', 'playerForTestSetAndFuturePreds']
        self.rosters_cols = ['playerId', 'teamId', 'status', 'date']
        self.salaries_cols = ['teamId', 'salary', 'year']
        self.standings_cols = ['teamId', 'wildCardRank', 'sportGamesBack', 'date']
        self.transactions_cols = ['playerId', 'transaction_flag', 'date']
        self.stat_cols = ["playerId", "target1_mean","target1_median","target1_std","target1_min","target1_max","target1_skew","target1_kurt",
                        "target2_mean","target2_median","target2_std","target2_min","target2_max","target2_skew","target2_kurt",
                        "target3_mean","target3_median","target3_std","target3_min","target3_max","target3_skew","target3_kurt",
                        "target4_mean","target4_median","target4_std","target4_min","target4_max","target4_skew","target4_kurt",
                        'tgt1_2_corr', 'tgt1_3_corr', 'tgt2_3_corr', 'tgt1_4_corr', 'tgt2_4_corr', 'tgt3_4_corr']

        self.scores_cols = ['playerId', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances', 'date']

        self.feature_cols1 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank', 'award_flag'] 

        self.feature_cols2 = ['label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank', 'award_flag'] 

        self.feature_cols3 = ['week_day','label_playerId', 'label_primaryPositionName', 'label_teamId',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank', 'diffmlbDebutDateflag', 'sincemlbDebutDateflag', 'award_flag'] 

        self.feature_cols4 = ['week_day', 'annual_day', 'month', 'label_playerId', 'label_primaryPositionName', 'label_teamId', 'label_birthCity',
                        'DOY', 'mlbDebutYear', 'DebutAge', 'heightInches', 'weight',
               'label_status', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
               'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
               'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
               'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
               'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
               'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
               'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
               'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
               'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
               'groundOutsPitching', 'runsPitching', 'doublesPitching',
               'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
               'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
               'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
               'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
               'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
               'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
               'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
               'inheritedRunnersScored', 'catchersInterferencePitching',
               'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
               'assists', 'putOuts', 'errors', 'chances',
                'season_info', 'wildCardRank', 'diffmlbDebutDateflag', 'sincemlbDebutDateflag', 'award_flag'] 
        
        
        # lightgbm
        self.params1 = {'objective':'mae',
                       'reg_alpha': 0.14947461820098767, 
                       'reg_lambda': 0.10185644384043743, 
                       'n_estimators': 3633, 
                       'learning_rate': 0.08046301304430488, 
                       'num_leaves': 674, 
                       'feature_fraction': 0.9101240539122566, 
                       'bagging_fraction': 0.9884451442950513, 
                       'bagging_freq': 8, 
                       'min_child_samples': 51}


        self.params2 = {'objective':'mae',
                       'reg_alpha': 0.1,
                       'reg_lambda': 0.1, 
                       'n_estimators': 80,
                       'learning_rate': 0.1,
                       'random_state': 42,
                       "num_leaves": 22}



        self.params3 = {'objective':'mae',
                       'reg_alpha': 0.1,
                       'reg_lambda': 0.1, 
                       'n_estimators': 10000,
                       'learning_rate': 0.1,
                       'random_state': 42,
                       "num_leaves": 100}

        self.params4 = {'objective':'mae',
                       'reg_alpha': 0.016468100279441976, 
                       'reg_lambda': 0.09128335764019105, 
                       'n_estimators': 9868, 
                       'learning_rate': 0.10528150510326864, 
                       'num_leaves': 157, 
                       'feature_fraction': 0.5419185713426886, 
                       'bagging_fraction': 0.2637405128936662, 
                       'bagging_freq': 19, 
                       'min_child_samples': 71}

    def make_feature(self, train_elements_dict):

        players = train_elements_dict['players']
        rosters = train_elements_dict['rosters']
        targets = train_elements_dict['targets']
        scores = train_elements_dict['scores']
        seasons = train_elements_dict['seasons']
        teams = train_elements_dict['teams']
        standings = train_elements_dict['standings']

        print('calc target stat ... ', end="")

        ## target stats
        targets_train = targets.merge(rosters[self.rosters_cols], on=['playerId', 'date'], how='left')
        targets_train = targets_train[(targets_train['date'] >= 20210601) & (targets_train['date'] < 20210701)]

        playerId_list = targets_train['playerId'].unique()
        player_target_probs = pd.DataFrame(columns = self.stat_cols)  
        for pid in tqdm(playerId_list):
            temp = targets_train[targets_train['playerId'] == pid]
            player_target_stats=calc_probs(pid,player_target_probs,temp)

        teamId_list = targets_train['teamId'].dropna().unique()
        team_target_probs = pd.DataFrame(columns = self.stat_cols)
        for pid in tqdm(teamId_list):
            temp = targets_train[targets_train['teamId'] == pid]
            team_target_stats=calc_probs(pid,team_target_probs,temp)

        team_stat_cols = self.stat_cols
        team_stat_cols = team_stat_cols[:1] + ["team_" + word for word in team_stat_cols[1:]]
        team_target_stats.columns = team_stat_cols

        self.feature_cols1 += self.stat_cols[1:-6]
        self.feature_cols2 += self.stat_cols[1:-6]
        self.feature_cols3 += self.stat_cols[1:-6]
        self.feature_cols4 += self.stat_cols[1:-6]

        self.feature_cols1 += team_stat_cols[1:]
        self.feature_cols2 += team_stat_cols[1:]
        self.feature_cols3 += team_stat_cols[1:]
        self.feature_cols4 += team_stat_cols[1:]

        print('done.')

        print('preprocess ... ', end="")
        ## salaries
        # salaries = salaries.groupby(['year', 'team']).sum()['salary'].reset_index()
        # salaries['team'] = salaries['team'].apply(map_team_name)
        # salaries = salaries.merge(teams, left_on='team', right_on='name', how='inner')
        # salaries = salaries.rename(columns={'id': 'teamId'})

        ## seasons
        seasons = seasons.fillna('0000-00-00')
        for c_ in seasons.columns[1:]:
            seasons[c_] = seasons[c_].str.replace('-', '').astype(int)

        ## players
        players['DOY'] = pd.to_datetime(players['DOB'], format="%Y-%m-%d").dt.year
        players['mlbDebutYear'] = pd.to_datetime(players['mlbDebutDate'], format="%Y-%m-%d").dt.year
        players['DebutAge'] = players['mlbDebutYear'] - players['DOY']
        players['mlbDebutDate'] = pd.to_numeric(players['mlbDebutDate'].str.replace('-', ''), errors="coerce")

        print('done.')

        print('creat feature ... ', end="")
        # creat feature
        train = targets[self.targets_cols].merge(players[self.players_cols], on=['playerId'], how='left')
        train = train.merge(rosters[self.rosters_cols], on=['playerId', 'date'], how='left')
        train = train.merge(scores[self.scores_cols], on=['playerId', 'date'], how='left')
        train = train.merge(player_target_stats, how='inner', left_on=["playerId"],right_on=["playerId"])
        train = train.merge(standings[self.standings_cols], on=['teamId', 'date'], how='left')
        train = train.merge(team_target_stats, how='left', left_on=["teamId"],right_on=["playerId"], suffixes=('', 'team_'))
        train = train.merge(awards, on=['playerId', 'date'], how='left')

        
        date_ = pd.to_datetime(train['date'], format="%Y%m%d")
        train['annual_day'] = (date_ - pd.to_datetime(date_.dt.year, format="%Y")) /  timedelta(days=1)
        train['week_day'] = date_.dt.weekday
        train['month'] = date_.dt.month
        train['year'] = date_.dt.year
        train['mlbDebutDateflag'] = (train['mlbDebutDate'] == train['date']) * 1
        train['sincemlbDebutDateflag'] = (train['date'] >= train['mlbDebutDate']) * 1
        train['diffmlbDebutDateflag'] = (train['date'] - train['mlbDebutDate'])

        # label encoding
        player2num = {c: i for i, c in enumerate(train['playerId'].unique())}
        position2num = {c: i for i, c in enumerate(train['primaryPositionName'].unique())}
        birthCityn2num = {c: i for i, c in enumerate(train['birthCity'].unique())}
        teamid2num = {c: i for i, c in enumerate(train['teamId'].unique())}
        status2num = {c: i for i, c in enumerate(train['status'].unique())}
        awardName2num = {c: i for i, c in enumerate(train['awardName'].unique())}
        train['label_playerId'] = train['playerId'].map(player2num)
        train['label_primaryPositionName'] = train['primaryPositionName'].map(position2num)
        train['label_birthCity'] = train['birthCity'].map(birthCityn2num)
        train['label_teamId'] = train['teamId'].map(teamid2num)
        train['label_status'] = train['status'].map(status2num)
        train['award_flag'] = train['awardSeason'].isna()*1


        ## season_info
        on_preseason_idxes = extract_season(train['date'], seasons[['preSeasonStartDate', 'preSeasonEndDate']])
        on_season_idxes = extract_season(train['date'], seasons[['regularSeasonStartDate', 'regularSeasonEndDate']]) * 2
        on_postseason_idxes = extract_season(train['date'], seasons[['postSeasonStartDate', 'postSeasonEndDate']]) * 3

        special_days = seasons['lastDate1stHalf'].to_list() + seasons['allStarDate'].to_list() + seasons['firstDate2ndHalf'].to_list()
        special_idxes = 0
        for day in special_days:
            special_idxes += (train['date'] == day) * 4

        on_total_season_idxes = on_preseason_idxes
        on_total_season_idxes[on_season_idxes==2] = 2
        on_total_season_idxes[on_postseason_idxes==3] = 3
        on_total_season_idxes[special_idxes==4] = 4

        train['season_info'] = on_total_season_idxes

        ## only on season
        on_whole_idxes = extract_season(train['date'], seasons[['seasonStartDate', 'seasonEndDate']])
        train = train[on_whole_idxes == 1].reset_index(drop=True)

        # train = train.merge(playerTwitterFollowers, how='left', on=["playerId", 'date'])

        ## only test_player
#         train = train[train['playerForTestSetAndFuturePreds']==True].reset_index(drop=True)

        print('done.')
        
        train_features_dict = {'players': players,
                               'seasons': seasons,
                                'player_target_stats': player_target_stats,
                                'team_target_stats': team_target_stats,
                                'player2num': player2num, 
                                'position2num': position2num, 
                                'birthCityn2num': birthCityn2num,
                                'teamid2num': teamid2num,
                                'status2num': status2num,
                                'feature_cols1': self.feature_cols1,
                                'feature_cols2': self.feature_cols2,
                                'feature_cols3': self.feature_cols3,
                                'feature_cols4': self.feature_cols4,
                                'rosters_cols_all': list(rosters.columns),
                                'scores_cols_all': list(scores.columns),
                                'standings_cols_all': list(standings.columns),
                                'awards_cols_all': list(awards.columns)
                              }
        
        if self.usetimelinefeature:
            ## game_info
            train['gameday'] = ~train['battingOrder'].isna()*1
            train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

            train=count_consecutive_items_n_cols(train,['playerId','gameday'],'daysSinceLastGame')
            train.loc[train['gameday']==1,'daysSinceLastGame'] = 0

            train_game = train[train['gameday']==1]
            train_last_game = train_game[~train_game.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
            train_last_game.columns = ['playerId', 'lastdate']
            train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
            train_last_game = pd.merge(train_player_unique, train_last_game, on=['playerId'], how='left' )
            train_last_game = train_last_game.fillna(20171231)
            
            train_features_dict['train_last_game'] = train_last_game
            self.feature_cols1 += ['daysSinceLastGame']
            self.feature_cols2 += ['daysSinceLastGame']
            self.feature_cols3 += ['daysSinceLastGame']
            self.feature_cols4 += ['daysSinceLastGame']
            
            
            ## rosters_info
            train['rosterday'] = ~train['status'].isna()*1
            train.sort_values(by=['playerId','date'],inplace=True,ascending=True)

            train=count_consecutive_items_n_cols(train,['playerId','rosterday'],'daysSinceLastRoster')
            train.loc[train['rosterday']==1,'daysSinceLastRoster'] = 0

            train_roster= train[train['rosterday']==1]
            train_last_roster = train_roster[~train_roster.duplicated(subset='playerId', keep='last')][['playerId', 'date']]
            train_last_roster.columns = ['playerId', 'lastroster']
            train_player_unique = pd.DataFrame(train['playerId'].unique(), columns=['playerId'])
            train_last_roster = pd.merge(train_player_unique, train_last_roster, on=['playerId'], how='left' )
            train_last_roster = train_last_roster.fillna(20171231)
            
            train_features_dict['train_last_roster'] = train_last_roster
            self.feature_cols1 += ['daysSinceLastRoster']
            self.feature_cols2 += ['daysSinceLastRoster']
            self.feature_cols3 += ['daysSinceLastRoster']
            self.feature_cols4 += ['daysSinceLastRoster']


        return train, train_features_dict
    
    def train_and_evaluate(self, train, isgamedayonly=False):
        
        if isgamedayonly:
            train = train[train['gameday'] == 1].reset_index(drop=True)
            
        train_X = train
        train_y = train[['target1', 'target2', 'target3', 'target4']]

        oof = np.zeros(train_y.shape) - 1.0
        y_valids = np.zeros(train_y.shape) - 1.0

        tr_idx = (train['date'].astype(int) < 20210401)
        val_idx = ~tr_idx

        x_train = train_X.loc[tr_idx].reset_index(drop=True)
        y_train = train_y.loc[tr_idx].reset_index(drop=True)
        x_valid = train_X.loc[val_idx].reset_index(drop=True)
        y_valid = train_y.loc[val_idx].reset_index(drop=True)

        study1 = optuna.create_study(direction='maximize')
        study1.optimize(fit_lgbm(x_train[self.feature_cols1], y_train['target1'], 
                                 x_valid[self.feature_cols1], y_valid['target1']), n_trials=100)
        
        study2 = optuna.create_study(direction='maximize')
        study2.optimize(fit_lgbm(x_train[self.feature_cols2], y_train['target2'], 
                                 x_valid[self.feature_cols2], y_valid['target2']), n_trials=100)
        
        study3 = optuna.create_study(direction='maximize')
        study3.optimize(fit_lgbm(x_train[self.feature_cols3], y_train['target3'], 
                                 x_valid[self.feature_cols3], y_valid['target3']), n_trials=100)
        
        study4 = optuna.create_study(direction='maximize')
        study4.optimize(fit_lgbm(x_train[self.feature_cols4], y_train['target4'], 
                                 x_valid[self.feature_cols4], y_valid['target4']), n_trials=100)

#         oof1, model1, score1 = fit_lgbm(
#             x_train[self.feature_cols1], y_train['target1'],
#             x_valid[self.feature_cols1], y_valid['target1'],
# #             self.params1
#         )
#         oof2, model2, score2 = fit_lgbm(
#             x_train[self.feature_cols2], y_train['target2'],
#             x_valid[self.feature_cols2], y_valid['target2'],
# #             self.params2
#         )
#         oof3, model3, score3 = fit_lgbm(
#             x_train[self.feature_cols3], y_train['target3'],
#             x_valid[self.feature_cols3], y_valid['target3'],
# #             self.params3
#         )
#         oof4, model4, score4 = fit_lgbm(
#             x_train[self.feature_cols4], y_train['target4'],
#             x_valid[self.feature_cols4], y_valid['target4'],
# #             self.params4
#         )

#         score = (score1+score2+score3+score4) / 4
#         print(f'score: {score}')

#         oof[val_idx, 0] = oof1
#         oof[val_idx, 1] = oof2
#         oof[val_idx, 2] = oof3
#         oof[val_idx, 3] = oof4
#         y_valids[val_idx, 0] = y_valid['target1'].values
#         y_valids[val_idx, 1] = y_valid['target2'].values
#         y_valids[val_idx, 2] = y_valid['target3'].values
#         y_valids[val_idx, 3] = y_valid['target4'].values

#         mae = mean_absolute_error(y_valids[val_idx, :], oof[val_idx, :])
#         print("mae:", mae)

#         val_idx_num = val_idx[val_idx==True].index.to_list()

#         oof_df = train[self.targets_cols]
#         oof_df.iloc[val_idx_num, 1:5] = oof[val_idx_num, :]

#         models = np.array([model1, model2, model3, model4])

        return study1, study2, study3, study4

In [17]:
rt4kaido_train = Rt4kaidoTrain(usetimelinefeature=True)
train, train_features_dict = rt4kaido_train.make_feature(train_elements_dict)

calc target stat ... 

100%|██████████| 30/30 [00:00<00:00, 42.08it/s]


done.
preprocess ... done.
creat feature ... done.


In [18]:
study1, study2, study3, study4 = rt4kaido_train.train_and_evaluate(train, isgamedayonly=True)

[I 2021-07-29 23:25:54,917] A new study created in memory with name: no-name-441d4c57-7254-4cdc-977d-b257866a6313


[LightGBM] [Warning] feature_fraction is set=0.9722869936637106, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9722869936637106
[LightGBM] [Warning] bagging_fraction is set=0.7758500817633951, subsample=0.5 will be ignored. Current value: bagging_fraction=0.7758500817633951
[LightGBM] [Warning] bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 2.53843	valid_1's l1: 2.56742
[200]	training's l1: 2.47203	valid_1's l1: 2.55732
[300]	training's l1: 2.41896	valid_1's l1: 2.55423


[I 2021-07-29 23:26:01,301] Trial 0 finished with value: -2.5530786201473004 and parameters: {'max_depth': 18, 'min_child_weight': 17, 'subsample': 0.5, 'colsample_bytree': 0.8, 'reg_lambda': 7.814945595626298, 'reg_alpha': 0.33299327270956247, 'feature_fraction': 0.9722869936637106, 'bagging_fraction': 0.7758500817633951, 'bagging_freq': 15, 'num_leaves': 117, 'min_child_samples': 48}. Best is trial 0 with value: -2.5530786201473004.


Early stopping, best iteration is:
[283]	training's l1: 2.42574	valid_1's l1: 2.55395
[LightGBM] [Warning] feature_fraction is set=0.24857453972189367, colsample_bytree=0.5 will be ignored. Current value: feature_fraction=0.24857453972189367
[LightGBM] [Warning] bagging_fraction is set=0.8492908445787666, subsample=0.9 will be ignored. Current value: bagging_fraction=0.8492908445787666
[LightGBM] [Warning] bagging_freq is set=15, subsample_freq=0 will be ignored. Current value: bagging_freq=15
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 2.51778	valid_1's l1: 2.59603
[200]	training's l1: 2.46589	valid_1's l1: 2.57559
[300]	training's l1: 2.44185	valid_1's l1: 2.57345
[400]	training's l1: 2.42358	valid_1's l1: 2.57067
[500]	training's l1: 2.40841	valid_1's l1: 2.57025
[600]	training's l1: 2.39573	valid_1's l1: 2.57015
Early stopping, best iteration is:
[527]	training's l1: 2.40525	valid_1's l1: 2.56916


[I 2021-07-29 23:26:12,292] Trial 1 finished with value: -2.5675261693533793 and parameters: {'max_depth': 20, 'min_child_weight': 13, 'subsample': 0.9, 'colsample_bytree': 0.5, 'reg_lambda': 13.027099736561231, 'reg_alpha': 25.45329004587263, 'feature_fraction': 0.24857453972189367, 'bagging_fraction': 0.8492908445787666, 'bagging_freq': 15, 'num_leaves': 702, 'min_child_samples': 33}. Best is trial 0 with value: -2.5530786201473004.


...

Early stopping, best iteration is:
[39]	training's l1: 2.0017	valid_1's l1: 2.14254
[LightGBM] [Warning] feature_fraction is set=0.9369567535309264, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9369567535309264
[LightGBM] [Warning] bagging_fraction is set=0.3476056186962182, subsample=0.7 will be ignored. Current value: bagging_fraction=0.3476056186962182
[LightGBM] [Warning] bagging_freq is set=20, subsample_freq=0 will be ignored. Current value: bagging_freq=20
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 1.74343	valid_1's l1: 2.20046


[I 2021-07-29 23:50:18,883] Trial 98 finished with value: -2.1435575656867205 and parameters: {'max_depth': 15, 'min_child_weight': 15, 'subsample': 0.7, 'colsample_bytree': 0.8, 'reg_lambda': 0.008574405591374566, 'reg_alpha': 6.786210816201581, 'feature_fraction': 0.9369567535309264, 'bagging_fraction': 0.3476056186962182, 'bagging_freq': 20, 'num_leaves': 258, 'min_child_samples': 38}. Best is trial 96 with value: -2.1212729444410936.


Early stopping, best iteration is:
[42]	training's l1: 1.87696	valid_1's l1: 2.14357
[LightGBM] [Warning] feature_fraction is set=0.9326949507326998, colsample_bytree=0.8 will be ignored. Current value: feature_fraction=0.9326949507326998
[LightGBM] [Warning] bagging_fraction is set=0.2902242561127369, subsample=0.9 will be ignored. Current value: bagging_fraction=0.2902242561127369
[LightGBM] [Warning] bagging_freq is set=19, subsample_freq=0 will be ignored. Current value: bagging_freq=19
Training until validation scores don't improve for 100 rounds
[100]	training's l1: 2.01502	valid_1's l1: 2.14004


[I 2021-07-29 23:50:20,416] Trial 99 finished with value: -2.132532518856557 and parameters: {'max_depth': 18, 'min_child_weight': 16, 'subsample': 0.9, 'colsample_bytree': 0.8, 'reg_lambda': 0.0029680902634028664, 'reg_alpha': 1.5458784189849764, 'feature_fraction': 0.9326949507326998, 'bagging_fraction': 0.2902242561127369, 'bagging_freq': 19, 'num_leaves': 25, 'min_child_samples': 52}. Best is trial 96 with value: -2.1212729444410936.


Early stopping, best iteration is:
[65]	training's l1: 2.06362	valid_1's l1: 2.1326


In [23]:
print(f'best_score = {-study1.best_value}')
study1.best_trial.params

best_score = 2.5426393658701856


{'max_depth': 6,
 'min_child_weight': 9,
 'subsample': 0.8,
 'colsample_bytree': 0.9,
 'reg_lambda': 0.08775848211046411,
 'reg_alpha': 0.0027773071193270436,
 'feature_fraction': 0.45190931022259706,
 'bagging_fraction': 0.5670215780108194,
 'bagging_freq': 13,
 'num_leaves': 107,
 'min_child_samples': 41}

In [24]:
print(f'best_score = {-study2.best_value}')
study2.best_trial.params

best_score = 2.8277580098012094


{'max_depth': 2,
 'min_child_weight': 5,
 'subsample': 0.7,
 'colsample_bytree': 0.8,
 'reg_lambda': 16.790678629980338,
 'reg_alpha': 0.009415230723916623,
 'feature_fraction': 0.9145403339290147,
 'bagging_fraction': 0.9513516876029571,
 'bagging_freq': 9,
 'num_leaves': 630,
 'min_child_samples': 51}

In [25]:
print(f'best_score = {-study3.best_value}')
study3.best_trial.params

best_score = 1.758952169856537


{'max_depth': 7,
 'min_child_weight': 4,
 'subsample': 0.5,
 'colsample_bytree': 0.6,
 'reg_lambda': 0.0010346852415822872,
 'reg_alpha': 0.014455742502665028,
 'feature_fraction': 0.5266195323088254,
 'bagging_fraction': 0.8791204332702245,
 'bagging_freq': 19,
 'num_leaves': 339,
 'min_child_samples': 76}

In [26]:
print(f'best_score = {-study4.best_value}')
study4.best_trial.params

best_score = 2.1212729444410936


{'max_depth': 16,
 'min_child_weight': 16,
 'subsample': 0.8,
 'colsample_bytree': 0.8,
 'reg_lambda': 0.009412366357658443,
 'reg_alpha': 3.169177163582572,
 'feature_fraction': 0.9345693637220724,
 'bagging_fraction': 0.37870852559215007,
 'bagging_freq': 19,
 'num_leaves': 31,
 'min_child_samples': 36}